In [1]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import glob

In [2]:
def clean(df, flag):

    col_train = ['start(exchange)[ms]', 'end(system)[ms]', 'end(exchange)[ms]',\
    'kinectstart(exchange)[ms]', 'kinectend(system)[ms]',\
    'kinectend(exchange)[ms]', 'SS_ternary', 'TC_ternary', 'TS_ternary', 'SS',\
    'TC1', 'TC2', 'TC3', 'TC4', 'TC5', 'TS1', 'TS2', 'TS3', 'TS4', 'TS5']
    col_test1 = ['Unnamed: 972', 'Unnamed: 971', 'Unnamed: 970', '無言', '名詁E形容詁E感動詁E副詁Elabel']
    col_test2 = ['adjective', 'no_utterance', 'adverbs', 'noun', 'interjection']

    df_col = df.columns.values 

    if flag:
        df = df.drop(col_train, axis=1) 
        df = df.drop(col_test1, axis=1) 
        df = df.drop(col_test2, axis=1)
    else:
        df = df.drop(col_train, axis=1) 
        if set(df_col) >= set(col_test1):
            df = df.drop(col_test1, axis=1)
        else:
            df = df.drop(col_test2, axis=1)
    return df.values 


In [3]:
def louocv(files, testfile):
    #train_dfが空かどうか（空のtrain_dfにconcat()できないための対応）
    flag = True

    for file in files:
        if file == testfile:
            test_df = pd.read_csv(file)
        else:
            if flag:
                train_df = pd.read_csv(file)
                flag = False 
            else:
                tmp_df = pd.read_csv(file)
                train_df = pd.concat([train_df, tmp_df])


    return train_df, test_df

In [12]:
FILE_PATH = "/home/ryoyanagimoto/Hazumi1902/dumpfiles/*.csv"
files = glob.glob(FILE_PATH)

test_preds = []
test_ans = []

for testfile in files:
    # train_df, test_df = louocv(files, testfile)

    # print(train_data.shape)

    # #train_data = train_df.drop(col, axis=1).values 
    # train_data = clean(train_df, True)
    # train_target = train_df['TS_ternary'].values 

    # print(train_data.shape)

    # #test_data = test_df.drop(col, axis=1).values 
    # test_data = clean(test_df, False)
    # test_target = test_df['TS_ternary'].values 

    # x_train, x_valid, y_train, y_valid = train_test_split(train_data, train_target, shuffle=True)

    iris = datasets.load_iris() 
    data = iris['data']
    target = iris['target'] 
    x_train, x_valid, y_train, y_valid = train_test_split(data, target, shuffle=True) 
    scaler = StandardScaler() 
    scaler.fit(x_train) 

    x_train = scaler.transform(x_train) 
    x_valid = scaler.transform(x_valid) 
    
    x_train = torch.from_numpy(x_train).to(torch.float)
    y_train = torch.from_numpy(y_train).to(torch.long)
    x_valid = torch.from_numpy(x_valid).to(torch.float)
    y_valid = torch.from_numpy(y_valid).to(torch.long)

    # x_test = torch.from_numpy(test_data).to(torch.float)
    # y_test = torch.from_numpy(test_target).to(torch.long)

    train_dataset = TensorDataset(x_train, y_train)
    valid_dataset = TensorDataset(x_valid, y_valid)

    batch_size = 1
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

    class Net(nn.Module):    
        def __init__(self):
            super(Net, self).__init__()
            self.fc1 = nn.Linear(4, 50)
            self.fc2 = nn.Linear(50, 3)
    
        def forward(self, x):
            x = self.fc1(x)
            x = F.relu(x)
            x = self.fc2(x)
            x = F.softmax(x, dim=1)
            return x

    net = Net()

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=0.01)

    # エポック数
    num_epochs = 5

    # 学習時と検証時で分けるためディクショナリを用意
    dataloaders_dict = {
        'train': train_dataloader,
        'val': valid_dataloader
    }

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')
        
        for phase in ['train', 'val']:
            
            if phase == 'train':
                # モデルを訓練モードに設定
                net.train()
            else:
                # モデルを推論モードに設定
                net.eval()
            
            # 損失和
            epoch_loss = 0.0
            # 正解数
            epoch_corrects = 0
            
            # DataLoaderからデータをバッチごとに取り出す
            for inputs, labels in dataloaders_dict[phase]:
                
                # optimizerの初期化
                optimizer.zero_grad()
                
                # 学習時のみ勾配を計算させる設定にする
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)
                    
                    # 損失を計算
                    loss = criterion(outputs, labels)
                    
                    # ラベルを予測
                    _, preds = torch.max(outputs, 1)
                    
                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        # 逆伝搬の計算
                        loss.backward()
                        # パラメータの更新
                        optimizer.step()
                    
                    # イテレーション結果の計算
                    # lossの合計を更新
                    # PyTorchの仕様上各バッチ内での平均のlossが計算される。
                    # データ数を掛けることで平均から合計に変換をしている。
                    # 損失和は「全データの損失/データ数」で計算されるため、
                    # 平均のままだと損失和を求めることができないため。
                    epoch_loss += loss.item() * inputs.size(0)
                    
                    # 正解数の合計を更新
                    epoch_corrects += torch.sum(preds == labels.data)

            # epochごとのlossと正解率を表示
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

    #pred = net(x_test)
    #test_ans = test_ans + test_target.tolist()
    #test_preds = test_preds + torch.argmax(pred, dim=1).tolist()

Epoch 1/5
-------------
train Loss: 0.9833 Acc: 0.6429
val Loss: 0.9014 Acc: 0.7368
Epoch 2/5
-------------
train Loss: 0.8523 Acc: 0.7500
val Loss: 0.8339 Acc: 0.7895
Epoch 3/5
-------------
train Loss: 0.8070 Acc: 0.7946
val Loss: 0.8004 Acc: 0.8421
Epoch 4/5
-------------
train Loss: 0.7813 Acc: 0.8125
val Loss: 0.7770 Acc: 0.8947
Epoch 5/5
-------------
train Loss: 0.7633 Acc: 0.8125
val Loss: 0.7594 Acc: 0.8947
Epoch 1/5
-------------
train Loss: 1.0395 Acc: 0.5982
val Loss: 0.9470 Acc: 0.6842
Epoch 2/5
-------------
train Loss: 0.9164 Acc: 0.6607
val Loss: 0.8573 Acc: 0.6842
Epoch 3/5
-------------
train Loss: 0.8551 Acc: 0.6875
val Loss: 0.8099 Acc: 0.7895
Epoch 4/5
-------------
train Loss: 0.8206 Acc: 0.7768
val Loss: 0.7787 Acc: 0.8158
Epoch 5/5
-------------
train Loss: 0.7965 Acc: 0.8125
val Loss: 0.7549 Acc: 0.8421
Epoch 1/5
-------------
train Loss: 1.0563 Acc: 0.5446
val Loss: 0.9910 Acc: 0.7632
Epoch 2/5
-------------
train Loss: 0.9541 Acc: 0.7857
val Loss: 0.8762 Acc:

In [8]:
from sklearn import datasets 
from sklearn.preprocessing import StandardScaler

In [ ]:
count = 0

for i in range(len(test_preds)):
    if test_preds[i] == test_ans[i]:
        count += 1

count / len(test_preds)

0.3714163457424048

In [ ]:
print(test_preds)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 